# 深度卷积模型：案例分析

## 1. 案例分析

### 1.1 为什么要进行案例分析

过去几年来，计算机视觉领域的主要工作，就在研究如何拼装卷积层、池化层、全连接层等等卷积神经网络模型的基本组件，组合出较为有效的模型，有效的模型通常可以在不同的计算机视觉模型之间通用。因此，通过学习这些模型实例，可以对深度卷积模型有更好的理解。

### 1.2 经典网络

LeNet-5，用来处理灰度图片的数字识别问题。
    - 在LeNet-5的年代，补全的技巧还不常用，所有卷积层都不补全
    - 池化层在当时更多采用的是平均池化，目前最大池化更为常见
    - 模型大约有60000个参数，相对于现代神经网络，规模较小
    - 随着模型的深入，高度和宽度不断减小，通道数不断增加，这个实践沿用至今
    - 当时的激活函数采用的是sigmoid或者tanh，还没有使用ReLU
    - 当时单个过滤器，并不会处理所有的通道，这点和现在也不一样
    - 当时在池化层后面会接一个非线性的激活层，和现在不同

![LeNet-5.png](img/LeNet-5.png)

AlexNet，真正使得计算机视觉领域开始重视神经网络
    - 已经开始使用ReLU作为激活函数
    - 大约6000万的参数，中等规模
    - 当时的GPU计算能力还不够强，论文里涉及两个GPU的通信
    - 用到了Local Response Normalization(LRN)的技巧，这个技巧目前已经不常用了
    - same padding，同一补全

![AlexNet.png](img/AlexNet.png)

VGG-16，模式更为明确
    - 固定3×3的过滤器，步长s=1，同一补全；最大池化层2×2，步长s=2
    - 每次卷积层（连续两层）后，通道数翻倍
    - 每次池化层后，高度和宽度减半
    - 大约有1.38亿的参数，现代规模的神经网络
    - VGG-16中的16是值整个神经网路中共有16层带有参数的层，另外还有一个更大的VGG-19版本，不常用

![VGG-16.png](img/VGG-16.png)

### 1.3 残差网络 ResNets

由于梯度消失和梯度爆炸问题，训练非常大的神经网络通常非常困难。而残差网络解决了这个问题。残差网络的基本组成结构如下：
![Residual block.png](img/Residual block.png)

由于计算能力的问题，普通神经网络（Plain Network）在实践中随着层数增多，并不能获得更好的效果。而残差神经网络使训练层数非常多的模型成为可能。
![Residual Network.png](img/Residual Network.png)

### 1.4 残差网络为什么有效

假设现在有一个 $l$ 层的普通神经网络，输出结果 $a^{[l]}$。我们在其之后，又增加了两层残差Block，得到输出 $a^{[l+2]}$，根据上面的公式，$a^{[l+2]}=g(z^{[l+2]}+a^{[l]})=g(W^{[l+2]}a^{[l+1]}+b^{[l+2]}+a^{[l]})$。

假设我们对整个网络进行了L2正则化，那么$W^{[l+2]}$ 和 $b^{[l+2]}$ 就会约等于0，使得 $a^{[l+2]}$ 约等于 $a^{[l]}$。也即是说，在最坏的情况下，增加了两层残差Block，也只会使得神经网络的输出相同。而较好的情况下，我们可以训练出更好的模型。

值得注意的是，$z^{[l+2]}$ 和 $a^{[l]}$ 要可加，这两个矩阵的维度需要相同。因此，残差网络通常配合着同一补全的卷积层来使用。而对于池化层，则会需要增加一个 $W_s$ 的矩阵，和 $a^{[l]}$ 相乘后再进行加运算。

### 1.5 网络中的网络，1×1卷积

1×1的卷积在单一通道下看似乎没什么用，但是当通道数多了之后，1×1的卷积实际上可以对跨通道之间的数据项进行非线性组合。
![Why does a 1×1 convolution do.png](img/Why does a 1×1 convolution do.png)

1×1的卷积还可以对通道数进行缩减（就像池化层对高度和宽度进行缩减），当然如果愿意的化，1×1卷积也可以增加通道数。
![Using 1×1 convolutions.png](img/Using 1×1 convolutions.png)

### 1.6 Inception网络的设想

避免考虑要用1×1的卷积，还是3×3的卷积，还是池化层。直接将所有这些，叠加到同一层网络中，让模型自己去学习参数。

![Motivation for inception network.png](img/Motivation for inception network.png)

5×5的卷积，可能会引入比较大的计算量。这时，使用1×1的卷积，在中间做一层瓶颈层，可以有效地降低计算量。

![The problem of computational cost.png](img/The problem of computational cost.png)

![Using 1×1 convolution.png](img/Using 1×1 convolution.png)

### 1.7 Inception网络

Inception模块使用到了上面的组件，对于3×3和5×5这样的卷积层，会在之前加入一层1×1的卷积作为瓶颈层，减少计算；而对于池化层，除去同一补全之外，还会在之后增加一层1×1的卷积层，用来缩减通道数量。

![Inception module.png](img/Inception module.png)

Inception网络是由多个Inception模块组合而成的。
![Inception network.png](img/Inception network.png)

一个小彩蛋，Inception的概念，其实就来自盗梦空间。
![WE NEED TO GO DEEPER.png](img/WE NEED TO GO DEEPER.png)

## 2. 在实际项目中使用卷积网络的一些建议

### 2.1 使用开源实现

上面介绍的神经网络架构都比较复杂，在实现的过程中，有很多需要注意的小技巧。对于计算机视觉应用，想要使用上面或其它研究文献中介绍的神经网络架构，通常的建议是去（比如Github上）寻找开源实现，在此基础上进行开发。

### 2.2 迁移学习

很多开源实现除去实现神经网络架构之外，还会包含该架构在知名数据集（比如ImageNet）上训练完成后的各项权重。直接使用这些权重作为预训练好的权重，在此之上进行迁移学习，而不是重新随机初始化权重从头进行训练，是一个比较好的实践。根据所要解决问题数据量的不同，迁移学习的方案也有几种不同的模式：

    - 如果训练集数量很小，建议冻结下载好的权重，在此基础上直接替换增加一层自己的Softmax分类层，只训练这个分类层的权重。冻结权重这个功能，各个框架都有支持；
    - 如果训练集数量有中等规模，可以相应地冻结更少层权重，不冻结的几层和自己的Softmax层用来训练；
    - 如果训练集数量非常大，可以只用下载的权重替代权重的随机初始化，替换Softmax，整个网络进行训练。

计算机视觉领域，迁移学习几乎是必然的选择。

### 2.3 数据扩增

对于计算机视觉领域，数据量总是显得不够，数据扩增是对原始数据进行加工，生成新数据的手段。

    - 镜像（左右转置），随机剪切（不完美，可能失去图像中的物体，但只要一张图随机剪切的子集够大，在实践中效果也会不错），旋转，修剪（Shearing，不常用），局部翘曲（Local Warping，不常用）。
    - 颜色偏移，比如对图片的RGB值进行较小幅度的变更。AlexNet论文中也介绍了应用PCA进行颜色偏移的方法。
    
实践中在实现时，可能会有一个单独的CPU线程，将扩增后的数据，喂给正常训练的其它CPU线程或者GPU。

### 2.4 计算机视觉领域的现状

相比问题的复杂度，打标数据少，手动设计的特征工程多，在神经网络的架构设计上比较讲究。

![Data vs hand-engineering.png](img/Data vs hand-engineering.png)

模型融合（Ensembling）和Multi-crop是两个在比赛中提升成绩的好办法，但生产环境中使用的很少。

![Tips for doing well on benchmarks or winning competitions.png](img/Tips for doing well on benchmarks or winning competitions.png)

## 3. Show me the Code

### 3.1 Keras教程 - 快乐之家

在本节，我们将会学习：
1. Python编写的高级神经网络API（编程框架）Keras，它可以跑在多个底层框架上，比如Tensorflow，CNTK。
2. 看看我们如何可以在几个小时内就构建一个深度学习算法。

开发Keras的目的，就是为了让深度学习工程师可以更快地构建和实验不同的模型。正如Tensorflow相对于Python标准库或者Numpy来说是更高级的框架，Keras是在其之上更高层次的框架，提供了更多的抽象。能够很快地将想法变成结果，对于找到正确的模型来说十分关键。但与此同时，Keras相比底层框架，限制也更多，所以会有一些十分复杂的模型，可以用Tensorflow来表示，但不能（轻易地）用Keras来表示。尽管如此，Keras对于绝大多数常见的模型来说，是十分管用的。

在这个练习中，我们会解决“快乐之家”难题，下面会有详细的解析。首先，让我们导入相应的模块。

In [1]:
import numpy as np
from keras import layers
from keras.layers import Input, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D
from keras.layers import AveragePooling2D, MaxPooling2D, Dropout, GlobalMaxPooling2D, GlobalAveragePooling2D
from keras.models import Model
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
import pydot
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from kt_utils import *

import keras.backend as K
K.set_image_data_format('channels_last')
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

%matplotlib inline

Using TensorFlow backend.


#### 3.1.1 快乐之家

下一个假期，你觉得和五位朋友一起过。快乐之家的地理位置十分便利，但最重要的是所有人都承诺，在家的时候会十分开心。所以，想进入房子的人必须证明他们现在十分开心。

<img src="img/happy-house.jpg" style="width:350px;height:270px;">
<caption><center> <u> <font color='purple'> **Figure 1** </u><font color='purple'>  : **the Happy House**</center></caption>

作为深度学习专家，为了保证“快乐”的原则被严格执行，你会构建一个算法，使用门口摄像头拍摄的图片来判断访客是否快乐。只有在判断访问快乐的时候，门才会自动打开。

你收集了用门口摄像头拍摄的一组朋友和自己的照片，数据集已经进行了打标。

<img src="img/house-members.png" style="width:550px;height:250px;">

执行下面的代码，对数据集进行正规化，并了解数据集的维度。

In [2]:
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = load_dataset()

# Normalize image vectors
X_train = X_train_orig/255.
X_test = X_test_orig/255.

# Reshape
Y_train = Y_train_orig.T
Y_test = Y_test_orig.T

print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))

number of training examples = 600
number of test examples = 150
X_train shape: (600, 64, 64, 3)
Y_train shape: (600, 1)
X_test shape: (150, 64, 64, 3)
Y_test shape: (150, 1)


**"快乐之家"数据集详情**:
- 图片维度 (64,64,3)
- 训练集: 600 张图片
- 测试集: 150 张图片

### 3.2 使用Keras构建模型

Keras非常适合用来做快速原型，在非常短的时间内，我们就可以构建出效果非常好的模型。

下面是Keras模型的一个例子：
```python
def model(input_shape):
    # Define the input placeholder as a tensor with shape input_shape. Think of this as your input image!
    X_input = Input(input_shape)

    # Zero-Padding: pads the border of X_input with zeroes
    X = ZeroPadding2D((3, 3))(X_input)

    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(32, (7, 7), strides = (1, 1), name = 'conv0')(X)
    X = BatchNormalization(axis = 3, name = 'bn0')(X)
    X = Activation('relu')(X)

    # MAXPOOL
    X = MaxPooling2D((2, 2), name='max_pool')(X)

    # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
    X = Flatten()(X)
    X = Dense(1, activation='sigmoid', name='fc')(X)

    # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
    model = Model(inputs = X_input, outputs = X, name='HappyModel')
    
    return model
```

注意到，Keras使用了和Tensorflow或者numpy不太相同的变量命名风格。最主要的一点，它并没有随着前向传播过程创建并赋值一系列诸如 `X`, `Z1`, `A1`, `Z2`, `A2` 的变量。对于不同的层，Keras代码中仅仅是通过 `X = ...` 来重新对 `X` 赋值。换句话说，前向传播中的每个步骤，我们不断地将计算结果重新写回同一个变量 `X`。唯一的例外是 `X_input`，考虑后最后我们需要用 `model = Model(inputs = X_input, ...)` 来创建Keras模型，`X_input` 不会被覆盖。

**练习**：实现 `HappyModel()`。

In [3]:
# GRADED FUNCTION: HappyModel

def HappyModel(input_shape):
    """
    Implementation of the HappyModel.
    
    Arguments:
    input_shape -- shape of the images of the dataset

    Returns:
    model -- a Model() instance in Keras
    """
    
    ### START CODE HERE ###
    # Feel free to use the suggested outline in the text above to get started, and run through the whole
    # exercise (including the later portions of this notebook) once. The come back also try out other
    # network architectures as well. 
        # Define the input placeholder as a tensor with shape input_shape. Think of this as your input image!
    X_input = Input(input_shape)

    # Zero-Padding: pads the border of X_input with zeroes
    X = ZeroPadding2D((3, 3))(X_input)

    # CONV -> BN -> RELU Block applied to X
    X = Conv2D(32, (7, 7), strides = (1, 1), name = 'conv0')(X)
    X = BatchNormalization(axis = 3, name = 'bn0')(X)
    X = Activation('relu')(X)

    # MAXPOOL
    X = MaxPooling2D((2, 2), name='max_pool')(X)

    # FLATTEN X (means convert it to a vector) + FULLYCONNECTED
    X = Flatten()(X)
    X = Dense(1, activation='sigmoid', name='fc')(X)

    # Create model. This creates your Keras model instance, you'll use this instance to train/test the model.
    model = Model(inputs = X_input, outputs = X, name='HappyModel')
    
    ### END CODE HERE ###
    
    return model

我们创建了一个函数，来描述我们的模型。要训练和测试这个模型，在Keras中分为四步：
1. 调用上面的函数创建模型。
2. 调用 `model.compile(optimizer = "...", loss = "...", metrics = ["accuracy"])` 编译模型。
3. 调用 `model.fit(x = ..., y = ..., epochs = ..., batch_size = ...)` 训练模型。
4. 调用 `model.evaluate(x = ..., y = ...)` 测试模型。

如果你想了解关于 `model.compile()`, `model.fit()`, `model.evaluate()` 及其参数的更多信息，可以参考官方的[Keras文档](https://keras.io/models/model/).

**练习**: 实现步骤1，创建模型。

In [4]:
### START CODE HERE ### (1 line)
happyModel = HappyModel((X_train.shape[1], X_train.shape[2], X_train.shape[3]))
### END CODE HERE ###

**练习**: 实现步骤2，设置学习过程的相关参数以编译模型。请小心地选择 `compile()` 的三个参数。提示：快乐之家是一个二分类问题。

In [5]:
### START CODE HERE ### (1 line)
happyModel.compile(optimizer="Adam", loss="binary_crossentropy", metrics=["accuracy"])
### END CODE HERE ###

**练习**: 实现步骤3，训练模型。选择epochs和批次大小。

In [6]:
### START CODE HERE ### (1 line)
happyModel.fit(x=X_train, y=Y_train, epochs=40, batch_size=16)
### END CODE HERE ###

Epoch 1/40
600/600 [==============================] - 2s 3ms/step - loss: 1.6885 - acc: 0.6850
Epoch 2/40
600/600 [==============================] - 2s 3ms/step - loss: 0.4840 - acc: 0.8133
Epoch 3/40
600/600 [==============================] - 2s 3ms/step - loss: 0.1665 - acc: 0.9383
Epoch 4/40
600/600 [==============================] - 2s 3ms/step - loss: 0.2119 - acc: 0.9033
Epoch 5/40
600/600 [==============================] - 2s 3ms/step - loss: 0.1183 - acc: 0.9500
Epoch 6/40
600/600 [==============================] - 2s 3ms/step - loss: 0.1503 - acc: 0.9450
Epoch 7/40
600/600 [==============================] - 2s 3ms/step - loss: 0.1109 - acc: 0.9633
Epoch 8/40
600/600 [==============================] - 2s 3ms/step - loss: 0.1329 - acc: 0.9550
Epoch 9/40
600/600 [==============================] - 2s 3ms/step - loss: 0.0636 - acc: 0.9800
Epoch 10/40
600/600 [==============================] - 2s 3ms/step - loss: 0.0613 - acc: 0.9850
Epoch 11/40
600/600 [============================

注意这时如果重新执行 `fit()`，`model` 会使用之前已经学到的参数继续训练，而不是重新对参数初始化。

**练习**: 实现步骤4，即测试/评估模型。

In [7]:
### START CODE HERE ### (1 line)
preds = happyModel.evaluate(x=X_test, y=Y_test)
### END CODE HERE ###
print()
print ("Loss = " + str(preds[0]))
print ("Test Accuracy = " + str(preds[1]))

150/150 [==============================] - 0s 1ms/step

Loss = 0.0747678126891
Test Accuracy = 0.979999997616


如果 `happyModel()` 有效的话，这里我们会获得一个远高于随机值（50%）的准确率。

这里给一个参考值，**40 epochs，95%的测试准确率**（99%的训练准确率），微批的大小为16，使用adam优化器。

如果这里没能取得很好的准确率（80%以上），下面有一些策略可以考虑尝试一下。

- 尝试使用 CONV->BATCHNORM->RELU 这样的结构
```python
X = Conv2D(32, (3, 3), strides = (1, 1), name = 'conv0')(X)
X = BatchNormalization(axis = 3, name = 'bn0')(X)
X = Activation('relu')(X)
```
直到高度和宽度非常小，而通道数非常大（比如大约32）。这时候，立方体中很多有用的信息都编码在通道维度中。之后可以打平立方体，使用一个全连接层。
- 在上面的结构后，使用最大池化层。这可以用来降低高度和宽度的维度。
- 修改优化器，我们发现Adam在这个问题中比较好用。
- 如果模型跑不起来，用内存问题，尝试降低微批大小（12是一个不错的折中值）
- 跑更多轮次（epoch），知道训练集准确率进入平缓状态。

即便你的模型准确率已经不错，也还是可以试着修改参数，以获得更好的效果。

**注意**: 如果你对模型的超参进行调节，测试集实际上就变成了开发集，你的模型很可能会对测试集（开发集）过拟合。在本练习中，我们暂时不考虑这个问题。

### 3.3 Keras中其它一些有用的函数

下面有两个Keras中的特性，可能会比较有用：
- `model.summary()`: 以表格的形式打印出模型各层输入输出的大小。
- `plot_model()`: 打印模型图片，甚至可以使用SVG()函数保存为".png"形式。

执行下面的代码。

In [8]:
happyModel.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 3)         0         
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 70, 70, 3)         0         
_________________________________________________________________
conv0 (Conv2D)               (None, 64, 64, 32)        4736      
_________________________________________________________________
bn0 (BatchNormalization)     (None, 64, 64, 32)        128       
_________________________________________________________________
activation_1 (Activation)    (None, 64, 64, 32)        0         
_________________________________________________________________
max_pool (MaxPooling2D)      (None, 32, 32, 32)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 32768)             0         
__________